# #11a Joint Elements (Points)
<i>Create joints to simulate connections either to other elements or with a boundary</i>
***

### Joints for points

In [100]:
import sys; sys.path.append('../') # Reference modules in parent directory

from LPI_22_0 import *
lusas = get_lusas_modeller()

# if not lusas.existsDatabase():
#     raise Exception("A model must be open before running this code")

import m100_Tools_And_Helpers.Helpers as Helpers
Helpers.initialise(lusas)

# Create a new model
lusas.newProject("Structural", "Joint for points.mdl")
# Get a reference to the model database
db = lusas.getDatabase()
# Set the analysis category & vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")
lusas.view().setIsometric()
lusas.view().showCoordSystem(False)

db.createLoadcase("")
db.createLoadcase("")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFLoadcase instance at 0x1678892708112>

### Supports

In [101]:
support_attr = db.createSupportStructural("Sup1")
support_attr.setStructural("R", "R", "R", "R", "R", "R", "F", "F", "C", "F")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFSupportStructural instance at 0x1678895115744>

In [102]:
point_load_x_attr = db.createLoadingConcentrated("Point Load X").setConcentrated(1,0,0,0,0,0)
point_load_y_attr = db.createLoadingConcentrated("Point Load Y").setConcentrated(0,1,0,0,0,0)
point_load_z_attr = db.createLoadingConcentrated("Point Load Z").setConcentrated(0,0,1,0,0,0)
loads = [point_load_x_attr, point_load_y_attr, point_load_z_attr]

### Joint mesh attributes

In [ ]:
# Joints assigned between two separate features
mesh_pinned = db.createPointMeshElementAttr("Point Mesh Pinned", "JNT4")
mesh_moment = db.createPointMeshElementAttr("Point Mesh Moments", "JSH4")

### Joint Materials

In [104]:
pinned_material_attr = db.createSpringJointMaterial("Pinned Material", [100, 100, 100]).setValue("Assignment", "Point")
moment_material_attr = db.createSpringJointMaterial("Moments material", [100, 100, 100, 100, 100, 100]).setValue("Assignment", "Point")

### Joint Geometry

In [ ]:
# Use joint length to determine eccentricity
geom_by_length_attr = db.createGeometricJoint("Geom by length").setJoint(0.0, 0.5, 0.5, True)
# known eccentricty of 0.5, regardless of joint length/arrangement
geom_ecc_z_attr     = db.createGeometricJoint("Geom ecc z").setJoint(0.5, 0.0, 0.0, False)

### Joints between two points

In [106]:
lower_points:list[IFPoint]=[]
upper_points:list[IFPoint]=[]

for x in [0, 1]:
    lower_points.append(Helpers.create_point(x, 0, 0))
    upper_points.append(Helpers.create_point(x, 0, 1))

In [107]:
support_attr.assignTo(lower_points)
pinned_material_attr.assignTo(lower_points[0])
moment_material_attr.assignTo(lower_points[1])

mesh_pinned.assignTo([lower_points[0], upper_points[0]])
mesh_moment.assignTo([lower_points[1], upper_points[1]])

point_load_x_attr.assignTo(upper_points)
assign = lusas.assignment().setAllDefaults().setLoadset(2)
point_load_y_attr.assignTo(upper_points, assign)
assign = lusas.assignment().setAllDefaults().setLoadset(3)
point_load_z_attr.assignTo(upper_points, assign)


<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFObjectSet instance at 0x1678893680512>

### Eccentric joints

In [108]:
for i in range(2):
    analysis = db.createAnalysisStructural("", False)

    for j in range(3):
        loadcase = db.createLoadcase("", analysis.getName())
        assign = lusas.assignment().setAllDefaults().setLoadset(loadcase)
        loads[j].assignTo(upper_points, assign)

        if i == 0:
            geom_by_length_attr.assignTo(lower_points, assign)
        else:
            geom_ecc_z_attr.assignTo(lower_points, assign)


### Solve

In [109]:
db.resetMesh()
db.updateMesh()

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFDatabase instance at 0x1678895101616>

In [110]:
for analysis in db.getAnalyses():
   analysis.solve(True)
db.openAllResults(True, True)

lusas.view().insertValuesLayer()
lusas.view().values().setResults("Force/Moment - 3D Joint (JSH4,JL46)", "My")
lusas.view().values().setShowAtGauss()
lusas.view().mesh().showElementOrient(True)